##### Project: Opioid Exposed Infant Covariates
##### Investigator: Stephen Patrick, Sarah Loch
##### Programmers: Sander Su, Chris Guardo
##### Date Created: 01/17/23
##### Last Modified: 09/30/25

#### Notes:
This step comes after birth visit in the workspace pipeline. The notebook finds evidence of opioid exposure for infants during or after the initial birth inpatient visit 


In [0]:
%run "../Project_modules"

#### NICU admission : 
##### The infant in phenotype cohort with exposed to opioid

##### CPT codes: initial code
- 99222:	Initial hospital care, per day, for the evaluation and management of a patient, which requires these 3 key components: A comprehensive history; A comprehensive examination; and Medical decision making of moderate complexity...
- 99477:	Initial hospital care, per day, for the evaluation and management of the neonate, 28 days of age or younger, who requires intensive observation, frequent interventions, and other intensive care services...
- 99221:	Initial hospital care, per day, for the evaluation and management of a patient, which requires these 3 key components: A detailed or comprehensive history; A detailed or comprehensive examination; and Medical decision making that is straightforward...
- 99223:	Initial hospital care, per day, for the evaluation and management of a patient, which requires these 3 key components: A comprehensive history; A comprehensive examination; and Medical decision making of high complexity...
- 99460:	Initial hospital or birthing center care, per day, for evaluation and management of normal newborn...
- 99463:	Initial hospital or birthing center care, per day, for evaluation and management of normal newborn infant admitted and discharged on the same date...

##### subsequent code
- 99232: Subsequent hospital care, per day, for the evaluation and management of a patient, which requires at least 2 of these 3 key components: An expanded problem focused interval history; An expanded problem focused examination...
- 99480: Subsequent intensive care, per day, for the evaluation and management of the recovering infant (present body weight of 2501-5000 grams)
- 99233: Subsequent hospital care, per day, for the evaluation and management of a patient, which requires at least 2 of these 3 key components: A detailed interval history; A detailed examination; Medical decision making of high complexity...
- 99479: Subsequent intensive care, per day, for the evaluation and management of the recovering low birth weight infant (present body weight of 1500-2500 grams)
- 99231: Subsequent hospital care, per day, for the evaluation and management of a patient, which requires at least 2 of these 3 key components: A problem focused interval history; A problem focused examination...
- 99478: Subsequent intensive care, per day, for the evaluation and management of the recovering very low birth weight infant (present body weight less than 1500 grams);
- 99462: Subsequent hospital care, per day, for evaluation and management of normal newborn...

##### discharge code
- 99238: Hospital discharge day management; 30 minutes or less​
- 99239: Hospital discharge day management; more than 30 minutes;;

##### After chart review, decided to have CASE as the infant in the phenotype exposed to opioid and got: 
- 99477 initial, then 99480, 99479, 99478
- Control as the infant in the phenotype exposed to opioid,except the case
- Calculate the NICU stay length of case
- Calculate the birth hospital stay length for every infant in the phenotype exposed to opioid



In [0]:
phenotype_table_location = " ***Insert file location*** "
phenotype_table=spark.sql(f"SELECT * FROM {phenotype_table_location}")

##### cohort:  infant in phenotype cohort with exposed to opioid during hospitalization

In [0]:
bh_exposed_opioid_infant = get_baby_bh_cohort(phenotype_table,"opioid_exposed")
bh_exposed_opioid_infant.createOrReplaceTempView("bh_exposed_opioid_infant")
bh_exposed_opioid_infant.display()

##### Get the CPT codes for the birth hospitalization cohort

In [0]:
sql=f"""

     select person_id,baby_person_id,baby_grid,procedure_occurrence_id,procedure_date,procedure_source_value,birth_datetime,
     first_visit_start_date,first_visit_end_date from
   
     (select * from bh_exposed_opioid_infant) a
     inner join {proc_table} b
     on a.baby_person_id = b.person_id
     where first_visit_start_date <= procedure_date and procedure_date <= first_visit_end_date;

    """
cpt_exposed_opioid_infant= spark.sql(sql)
cpt_exposed_opioid_infant.createOrReplaceTempView("cpt_exposed_opioid_infant")

##### Validation

In [0]:
df_inspection("cpt_exposed_opioid_infant","baby")


##### Get the initial or subsequent CPT code records

In [0]:
code_list=['99222','99477','99221','99223','99232','99480','99233','99479','99231','99478','99460','99463','99462','99238','99239']
code_list_str=','.join(map(lambda x: "'" + x + "'", code_list))
           
sql=f"""
     select a.*,b.concept_name from 
     (select * from cpt_exposed_opioid_infant where procedure_source_value in ({code_list_str}) ) a
     left join (select * from {concept_table} where vocabulary_id = 'CPT4') b 
     on a.procedure_source_value = b.concept_code;
     """
subsequent_cpt_exposed_opioid_infant= spark.sql(sql)
subsequent_cpt_exposed_opioid_infant.createOrReplaceTempView("subsequent_cpt_exposed_opioid_infant")

#### Validation

In [0]:
df_inspection("subsequent_cpt_exposed_opioid_infant","baby")

In [0]:
sql="""
     select a.*, b.first_date_of_cpt99477 from 
     ( 
      select * from subsequent_cpt_exposed_opioid_infant
      where person_id in 
      
      (select person_id from cpt_exposed_opioid_infant
      where procedure_source_value in ('99477','99478','99479','99480'))
      ) a 
      
      left join 
     
      (select distinct person_id, min(procedure_date) as first_date_of_cpt99477
      from subsequent_cpt_exposed_opioid_infant where procedure_source_value = '99477' 
      group by person_id,procedure_date) b
      on a.person_id = b.person_id;
   """
subsequent_cpt_exposed_opioid_infant_case= spark.sql(sql)
subsequent_cpt_exposed_opioid_infant_case.createOrReplaceTempView("subsequent_cpt_exposed_opioid_infant_case")

##### Validation

In [0]:
df_inspection("subsequent_cpt_exposed_opioid_infant_case","baby")

In [0]:
sql=f"""
       select a.* from subsequent_cpt_exposed_opioid_infant_case a
       left join {phenotype_table} b
       on a.person_id = b.baby_person_id;
   """
subsequent_cpt_exposed_opioid_infant_case_grid= spark.sql(sql)
subsequent_cpt_exposed_opioid_infant_case_grid.createOrReplaceTempView("subsequent_cpt_exposed_opioid_infant_case_grid")

##### Calculating: nicu stay + hospital stay of case and control

In [0]:
sql="""
     select * from 
    
     (select baby_GRID, count(distinct procedure_date) as count_of_distinct_nicu_dates 
     from subsequent_cpt_exposed_opioid_infant_case_grid
     where procedure_source_value in ('99477','99478','99479','99480')
     group by baby_GRID) a
     
     right join
     
     (select a.baby_grid,b.first_visit_end_date,b.first_visit_start_date,DATEDIFF (b.first_visit_end_date, 
     b.first_visit_start_date) as length_of_hospital_stay from global_temp.mom_baby_step1_baby1stvisit a 
     inner join bh_exposed_opioid_infant b
     on a.baby_grid = b.baby_grid) b 
     
     using (baby_GRID);
   """
exposed_opioid_infant_nicu_hospital_stay = spark.sql(sql)
exposed_opioid_infant_nicu_hospital_stay.name='exposed_opioid_infant_nicu_hospital_stay'
register_parquet_global_view(exposed_opioid_infant_nicu_hospital_stay)

In [0]:
# %sql
# select * from global_temp.exposed_opioid_infant_nicu_hospital_stay;

##### Validation

In [0]:
# sql="""
#        select count(*) as total,count(distinct baby_grid) as unique_baby from 
#        global_temp.exposed_opioid_infant_nicu_hospital_stay;
#     """
# inspect_df = spark.sql(sql)
# inspect_df.display()

In [0]:
# sql="""
#        select count(*) as total,count(distinct baby_grid) as unique_baby from 
#        global_temp.exposed_opioid_infant_nicu_hospital_stay
#        where count_of_distinct_nicu_dates is not null;
#     """
# inspect_df = spark.sql(sql)
# inspect_df.display()

### Save Output for future use

In [0]:
exposed_opioid_infant_nicu_hospital_stay.write.mode("overwrite").saveAsTable(f"covariate_output.exposed_opioid_infant_nicu_hospital_stay")